In [64]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
df=pd.read_csv('chatInfo_rows.csv')

In [3]:
df.head()

,id,created_at,name,content,time
0,1,2024-11-02 13:48:55.289025+00,PEANUT,"Bonjour, premier message",13:16:44.321092
1,71,2024-11-16 22:19:00.679082+00,Okarun,Hey,22:19:00.679082
2,73,2024-11-16 22:24:39.5685+00,Chihiro,Heyoooooo,22:24:39.5685
3,76,2024-11-16 22:27:59.056803+00,JK,Hello,22:27:59.056803
4,77,2024-11-16 22:42:53.231892+00,Okarun,Hellooo,22:42:53.231892


In [30]:
((df['name'].str.strip().str.lower()=='jk')).sum()

np.int64(1)

In [41]:
df['msg_per_users']=df['name'].map(df['name'].value_counts())

In [42]:
df.head()

,id,created_at,name,content,time,msg_per_users
0,1,2024-11-02 13:48:55.289025+00,PEANUT,"Bonjour, premier message",13:16:44.321092,1
1,71,2024-11-16 22:19:00.679082+00,Okarun,Hey,22:19:00.679082,7
2,73,2024-11-16 22:24:39.5685+00,Chihiro,Heyoooooo,22:24:39.5685,1
3,76,2024-11-16 22:27:59.056803+00,JK,Hello,22:27:59.056803,1
4,77,2024-11-16 22:42:53.231892+00,Okarun,Hellooo,22:42:53.231892,7


In [79]:
df_sorted = df.sort_values(by="msg_per_users", ascending=False)

In [88]:
name_sorted=df_sorted[['name','msg_per_users']].drop_duplicates()

In [89]:
name_sorted.head()

,name,msg_per_users
1,Okarun,7
7,suley,5
27,Medh9,4
38,Hello,3
34,Suuu,2


In [65]:
conn = psycopg2.connect(
    host="127.0.0.1",
    port=5432,
    user="chatgouser",
    password="123",
    dbname="chatgo"
)

In [66]:
cur=conn.cursor()

In [69]:
# Récupérer le contenu pour vérifier
cur.execute("SELECT * FROM messages;")
rows = cur.fetchall()
for row in rows:
    print(row)

(1, 'Alice', 'Salut le monde !', datetime.datetime(2025, 10, 2, 22, 59, 57, 495665))
(2, 'moi', 'aurevoir !', datetime.datetime(2025, 10, 2, 23, 15, 41, 283571))


In [70]:
cur.execute("""
INSERT INTO messages (sender, content) VALUES (%s, %s)
""", ("moi", "aurevoir !"))

In [72]:
conn.commit()

In [99]:
conn.rollback()  # Annule la transaction en échec

In [100]:
# Avec un DataFrame
for index, row in name_sorted.iterrows():
    cur.execute("""
    INSERT INTO messages (sender, content) VALUES (%s, %s)
    """, (row['name'], row['msg_per_users']))

In [101]:
conn.commit()